In [7]:
import os
import filecmp
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_all_files(folder):
    """
    주어진 폴더 내의 모든 파일 경로를 재귀적으로 가져옵니다.
    """
    file_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

def compare_file_pair(file1, file2):
    """
    두 파일을 비교하여 이름, 크기, 내용이 동일한지 여부를 반환합니다.
    """
    if os.path.getsize(file1) != os.path.getsize(file2):
        return False
    return filecmp.cmp(file1, file2, shallow=False)

def compare_folders(folder1, folder2):
    # 폴더 내 모든 파일의 경로를 가져옵니다.
    files_folder1 = {os.path.relpath(path, folder1): path for path in get_all_files(folder1)}
    files_folder2 = {os.path.relpath(path, folder2): path for path in get_all_files(folder2)}

    # 두 폴더에서 동일한 상대 경로를 가진 파일을 찾습니다.
    common_files = set(files_folder1.keys()).intersection(files_folder2.keys())

    # 병렬 처리 설정
    comparison_results = {}
    with ThreadPoolExecutor() as executor:
        future_to_file = {executor.submit(compare_file_pair, files_folder1[relative_path], files_folder2[relative_path]): relative_path for relative_path in common_files}
        for future in as_completed(future_to_file):
            relative_path = future_to_file[future]
            try:
                is_same = future.result()
                comparison_results[relative_path] = is_same
            except Exception as exc:
                print(f"{relative_path} generated an exception: {exc}")

    return comparison_results

# 폴더 경로 설정
folder1 = r"D:\SamsungSTF\Processed_Data\Multi_TripByTrip"
folder2 = r"D:\SamsungSTF\Processed_Data\TripByTrip"

# 비교 수행
results = compare_folders(folder1, folder2)

# "Different" 파일만 출력
different_files_found = False
for file, is_same in results.items():
    if not is_same:
        print(f"{file} : Different")
        different_files_found = True

if not different_files_found:
    print("다른 파일이 없습니다.")


다른 파일이 없습니다.
